# OpenCVでとことん画像処理 - 前半

このノートは **前半** のノートになります。前半では

+ [画像を扱う際の簡単な基礎知識](#basic)
    + [画像処理のよく使われる場面など](#example)
    + [表色系と色空間](#color)
+ [画素ごとの濃淡変換](#contrast)
    + [トーンカーブ](#tonecurve)
    + [LUTによる高速濃淡変換](#LUT)
    + [(おまけ)salt\_pepperノイズ](#sp_noise)
+ [複数画像の利用](#multi_img)
    + [アルファブレンディング](#alpha)
    + [ディゾルブ](#dissolve)
    
を扱います。

※追記: Python 3.8.2, chromiumベースブラウザ で動作を確認しております

## モジュールのインポート

In [ ]:
%reload_ext autoreload
%autoreload 2

# これを書くとjupyterでmatplotlibなどが出力する画像の解像度が上がる
# %config InlineBackend.figure_format = 'retina'

import cv2
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import os
import sys
sys.path.append("../../")
import time

from utils.plot import compare_plot

print(cv2.__version__)
print(matplotlib.__version__)

In [ ]:
from tqdm import tqdm_notebook as tqdm

<a id="basic"></a>
## 画像を扱う際の簡単な知識

デジタル画像は、多数の画素で構成されていて、画素値を多数の数値データと考えることでデータの前処理を行うことができます。

In [ ]:
# まずは画像を読み込んでみましょう
img = cv2.imread("../../data/Lenna.png", 1)

In [ ]:
# matplotlibで画像を出力する場合はimshowを使用
plt.imshow(img)

In [ ]:
# 上記なままではBGRの順番で読み込む(色空間については後述)のでRGBへ変換します
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img)

In [ ]:
# 上のRGB画像をグレー画像にして読み込みます
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
plt.imshow(img_gray, cmap="gray")

In [ ]:
# 数値データで見てみます。カラーなので[red,green,blue]の色が0~255で格納されています。
img[0][0]

<a id="example"></a>
### 画像処理のよく使われる場面など

カラーの画像分類チュートリアルによく使用されるCIFAR-10データセットなども、上記のように数値で表現された画像から特徴を学習させていきます。
<img src="../fig/cifar10.png">

<a id="color"></a>
## 表色系と色空間

色を心理的な観点から**色相(Hue)**、**明度(Saturation)**、**彩度(Value)**の三属性で表す方法をマンセル表色系と呼びます。

+ 色相
    + 色の違いを示す属性
+ 明度
    + 各色相の明るさを表す属性
+ 彩度
    + 彩度は色の鮮やかさを示す属性
    
<img src="../fig/muncell_colormap.png">

一つの円の円周上で5等分するように配置します。色はR(赤), Y(黄), G(緑), B(青), P(紫)。

## RGB表色

CIE（国際照明委員会）が定める表色系。RGB表色系原色をR（赤、700nm）、G（緑、546.1nm）、B（青、435.8nm）とする表色系を、CIEのRGB表色系と呼んでいます。  

これらRGB表色系をベースとして画像処理を行っていきます。


<a id="contrast"></a>
## 画素ごとの濃淡変換
濃淡の変換は単純な処理ではありますが、画像の「**見栄え**」を変えたりするためなどに、使用される画像処理になります。  

酒直関数の画素値に対して、どうやって対応づけるかを指定することを階調変換といい、それをグラフで表したものを**トーンカーブ**と呼びます。

## ヒストグラム
画像のコントラストを上げたい時、まずは画像のヒストグラムを調べて分布を見ます。

In [ ]:
img[:,:,0] #R(赤)の画素値

In [ ]:
labels = ["RED", "GREEN", "BLUE"]
fig, ax = plt.subplots(1, 3, figsize=(18,6))
for i, rgb in enumerate(labels):
    img_rgb = img[:,:,i]
    ax[i].hist(img_rgb.ravel(),256,[0,256]);
    ax[i].set_title("img_{}".format(rgb))
    ax[i].set_xlabel("image pixel value")
    ax[i].set_ylabel("number of value")
plt.show()

## ヒストグラム平坦化
偏りが集中している場合はその画素をできるだけ広げて均等に分布することで、明るさの違う画像でもトーンカーブの結果を等しくすることができます。

In [ ]:
# 集中していた山がなだらかになります
labels = ["RED", "GREEN", "BLUE"]
fig, ax = plt.subplots(1, 3, figsize=(18,6))
img_rgb = []
for i, rgb in enumerate(labels):
    # ヒストグラムを平坦化する
    equ = cv2.equalizeHist(img[:,:,i])
    
    img_rgb.append(equ)
    ax[i].hist(equ.ravel(),256,[0,256]);
    ax[i].set_title("img_{}".format(rgb))
    ax[i].set_xlabel("image pixel value")
    ax[i].set_ylabel("number of value")
plt.show()

In [ ]:
img_hist = np.dstack((np.dstack((img_rgb[0], img_rgb[1])), img_rgb[2]))

In [ ]:
# 元画像とヒストグラム平坦化後の比較
compare_plot([img, img_hist], ["before", "after"])

In [ ]:
cv2.imwrite("../../data/Lenna_flat.png", cv2.cvtColor(img_hist, cv2.COLOR_RGB2BGR))

<a id="tonecurve"></a>
## トーンカーブ

### 折れ線型トーンカーブ
折れ線で表されるトーンカーブでは、カーブの設計が容易であるため実際によく用いられます。

In [ ]:
polyline = np.vectorize(lambda x: 0 if x < 64 else 2*(x-64) if (64 <= x <192) else 256)

In [ ]:
plt.plot([polyline(i) for i in range(256)])
plt.xlabel("input pixel")
plt.ylabel("output pixel")
plt.title("polyline graph")

In [ ]:
compare_plot([img_hist, polyline(img_hist).astype("uint8")], \
        ["before", "after"])

### S字トーンカーブ
画素値の127を境として、低い入力値(暗部)はより低い出力値に、高い入力値(明部)はより高い出力値に変換されるので、結果として暗部・明部が強調され、出力画像を見るとコントラストが高くなっています。

In [ ]:
s_shape_curve = np.vectorize(lambda x: (np.sin(np.pi * (x/255 - 0.5)) + 1)/2 * 255)

In [ ]:
plt.plot([s_shape_curve(i) for i in range(256)])
plt.xlabel("input pixel")
plt.ylabel("output pixel")
plt.title("S shape curve graph")

In [ ]:
compare_plot([img_hist, s_shape_curve(img_hist).astype("uint8")], \
            ["before", "after"])

### ガンマ変換

ガンマ変換（ガンマ補正）とは下記の関数に則って画素値を変換する手法になっています。

$$ y = 255 \cdot (\frac{x}{255})^{\frac{1}{\gamma}} $$

画素値を明るくしたい場合にはガンマの値を1より大きくすることで明るくすることができ、逆に暗くしたい場合にはガンマの値を1より小さくすることで暗くすることができます。

In [ ]:
def curve_gamma(x, gamma):
    return 255 * (x/255) ** (1/gamma)

g_curve_light = np.vectorize(lambda x: curve_gamma(x, 2))
g_curve_dark= np.vectorize(lambda x: curve_gamma(x, 0.5))

In [ ]:
labels = ["gamma curve (gamma=2)", "gamma curve (gamma=0.5)"]
fig, ax = plt.subplots(1, 2, figsize=(18,6))
for i, graph in enumerate([[g_curve_light(i) for i in range(256)], [g_curve_dark(i) for i in range(256)]]):
    ax[i].plot(graph)
    ax[i].set_title(labels[i])
plt.show()

In [ ]:
# 画像を明るくするガンマ変換
compare_plot([img_hist, g_curve_light(img_hist).astype("uint8")], \
            ["before", "after"])

In [ ]:
# 画像を暗くするガンマ変換
compare_plot([img_hist, g_curve_dark(img_hist).astype("uint8")], \
            ["before", "after"])

<a id="LUT"></a>
## LUT(Look Up Table)による高速濃淡変換
ルックアップテーブルとは複雑な画像処理を単純なリストや配列の参照処理で置き換えて効率化を図るために作られたもので、これによって画像処理を高速化することができます。

例えば、フルHD(1920×1080)であれば2073600個の画素があります。フルHDの画像の階調変換を一回行おうとすると、素朴に考えて変換の計算を2073600回行わねばならないことになります。RGB各チャンネルに変換処理を行うとその3倍となります。

しかしよく考えると、入力も出力も、取りうる値は 0から255 の整数となります。画素値変換の対応は 256 通りなので、変換の計算を行わずに、「0から255 の各入力値に対して、出力値が 0から255 のうちどれに対応するか」 を示す表をあらかじめ作っておいて、変換時は画素値の計算をすることで高速に処理ができます。表の作成自体は256回の計算で済むので、単調に計算処理をするより早くなるというメカニズムです。

### ネガポジ反転

In [ ]:
def neg_pos_rev_LUT(img):
    look_up_table = np.zeros((256, 1), dtype = 'uint8') 
    for i in range(256):
        look_up_table[i][0] = 255 - i
    img_np_rev_LUT = cv2.LUT(img, look_up_table)
    return img_np_rev_LUT

In [ ]:
compare_plot([img_hist, neg_pos_rev_LUT(img_hist)], \
            ["before", "after"])

### 速度比較
実際にLUTと通常の画素値変換の速度を比較してみようと思います。

In [ ]:
neg_pos_rev = np.vectorize(lambda x: x * (-1))

In [ ]:
plt.plot([neg_pos_rev(i) for i in range(256)])
plt.xlabel("input pixel")
plt.ylabel("output pixel")
plt.title("negative positive reverse")

In [ ]:
compare_plot([img_hist, neg_pos_rev(img_hist).astype("uint8")], \
            ["before", "after"])

In [ ]:
# 通常の画素値変換を計測
tic = time.time()
neg_pos_rev(img_hist)
toc = time.time()
print("Execution Time: {} sec".format(toc - tic))

In [ ]:
# LUTを使ったテーブル変換を計測
tic = time.time()
neg_pos_rev_LUT(img_hist)
toc = time.time()
print("Execution Time: {} sec".format(toc - tic))

LUTを使った速度のほうが約80倍近く高速化できていることがわかります。

### ポスタリゼーション(n値化)
画素値を数段階に制限して出力するような変換をポスタリゼーションと呼んでおり、さらに出力値を2段階に制限している処理は特に2値化(バイナリゼーション)と呼んでいます。

In [ ]:
def posterize_LUT(img, n):
    look_up_table = np.zeros((256, 1), dtype = 'uint8') 
    for i in range(256):
        look_up_table[i][0] = np.floor(i/(256/n+1)) * 256/(n)
    img_np_rev_LUT = cv2.LUT(img, look_up_table)
    return img_np_rev_LUT

In [ ]:
posterize = lambda x, n: np.floor(x/(256/n+1)) * 256/(n)
plt.plot([posterize(i,256) for i in range(256)])
plt.xlabel("input pixel")
plt.ylabel("output pixel")
plt.title("posterization")

In [ ]:
# 階調を4段階に(2の2乗)して変換
compare_plot([img_hist, posterize_LUT(img_hist, 3).astype("uint8")], \
            ["before", "after"])

ポスタリゼーションは特にグレー画像にていい感じに効果を発揮することが多いですが、上記のようなRGB画像のような画像では段階を複雑にするほど色の組み合わせが増えるため、うまくいかないことが多い傾向にあります。

In [ ]:
# グレー画像を2値化
compare_plot([img_gray, posterize_LUT(img_gray, 6).astype("uint8")], \
            ["before", "after"],
            ["gray", "gray"])

<a id="sp_noise"></a>
## (おまけ)ソルト・ペッパーノイズ
塩と胡椒をかけたようなノイズなので一般的にはこう呼ばれており、インパルスノイズとも呼ばれるものです。

In [ ]:
def sp_noize(img, sp, amount):
    row,col,ch = img.shape
    s_vs_p = sp # 塩と胡椒の割合
    amount = amount # 画像の何割をノイズ化するか
    sp_img = img.copy()

    # salt
    num_salt = np.ceil(amount * img.size * s_vs_p)
    coords = [np.random.randint(0, i-1 , int(num_salt)) for i in img.shape]
    sp_img[coords[:-1]] = (255,255,255)

    # pepper
    num_pepper = np.ceil(amount* img.size * (1. - s_vs_p))
    coords = [np.random.randint(0, i-1 , int(num_pepper)) for i in img.shape]
    sp_img[coords[:-1]] = (0,0,0)
    
    return sp_img

In [ ]:
compare_plot([img_hist, sp_noize(img_hist, 0.5, 0.005).astype("uint8")], \
            ["before", "after"])

<a id="multi_img"></a>
## 複数画像の利用
これまでの画像処理は1枚の画像に対する処理でしたが、複数枚用いるような画像処理についても紹介していきたいと思います。複数画像用にもう1枚画像を読み込んでいきます。

In [ ]:
img_cat = cv2.cvtColor(cv2.imread("../../data/kuroneko.png"), cv2.COLOR_BGR2RGB)

<a id="alpha"></a>
## アルファブレンディング
アルファブレンディングとは2枚の画像を重みを基準にして合成したような画像を作成する処理のことを指しています。具体的に式にしてみると以下のように表せます。

$$ g = \alpha \cdot f_1 + (1 - \alpha) \cdot f_2 $$

In [ ]:
def alpha_blend(img1, img2, alpha):
    return cv2.addWeighted(img1, alpha, img2,(1-alpha), 0) 

In [ ]:
compare_plot([img_hist, img_cat, alpha_blend(img_hist, img_cat, 0.5)], \
            ["img1", "img2", "alpha_blending"])

<a id="dissolve"></a>
## ディゾルブ
アルファブレンディングを応用した例の一つにディゾルブという手法があり、アルファを時間的に変化させることで、あるシーンから別のシーンに徐々に変換していくような処理をすることができます。

In [ ]:
# ディゾルブ(時間ごとにαが変化して画像が切り替わります)
if os.path.isfile('../../data/dissolve.mp4'):
    os.remove('../../data/dissolve.mp4')

# videoの縦横サイズは画像と一致しないとうまく書き出せないので注意
fourcc = cv2.VideoWriter_fourcc(*'H264')
video = cv2.VideoWriter('../../data/dissolve.mp4', fourcc, 30.0, (512, 512))

for ratio in range(1, 101):
    img_alpha_blend = alpha_blend(img_hist, img_cat, ratio*0.01)
    video.write(cv2.cvtColor(img_alpha_blend, cv2.COLOR_RGB2BGR))
video.release()

In [ ]:
from IPython.display import Video
Video("../../data/dissolve.mp4")

## 参考文献・サイト

+ ディジタル画像処理 改訂新版 2~4章
+ トーンカーブ と LUT を理解する実装実験
> http://optie.hatenablog.com/entry/2018/03/03/141427